In [1]:
# Libraries
import zipfile
import os
import polars as pl
import pandas as pd
from pathlib import Path
from tqdm import tqdm

### Helper Functions

In [ ]:
# ### Opening the zip file
# import zipfile

# zip_path = "/Users/lex/CodeProjects/MyProject/Mitaron/Data/Child Medical Expense Dataset Oct 2025.zip"
# password = b"keio20251028"

# with zipfile.ZipFile(zip_path) as zf:
#     zf.extractall(pwd=password)

In [ ]:
# 一つのcsvをparquetに
from pathlib import Path

def csv_to_parquet(csv_path, parquet_folder = "/Users/lex/CodeProjects/MyProject/Mitaron/Data/parquete_data"):
    """
    Converting a CSV file to parquet and saving it to folder

    Args:
        file_path (_type_): path to an initial .csv file
        parquet_folder: folder where Parquet files will be saved

    Returns: 
        -Polars DataFrame¥
    """
    csv_path = Path(csv_path)
    parquet_folder = Path(parquet_folder)

    print(f"Reading: {csv_path}")
    df = pl.read_csv(csv_path, encoding="cp932")

    # Creating parquet file name
    parquet_file = parquet_folder / csv_path.with_suffix('.parquet').name

    # Saving as parquet
    df.write_parquet(parquet_file)
    print(f"✅ Saved: {parquet_file}")

    return df

In [5]:
# フォルダー全体をCSV -> parquet
def convert_folder_to_parquet_with_progress(csv_folder, parquet_folder="/Users/lex/CodeProjects/MyProject/Mitaron/Data/parquete_data"):
    """
    Convert all CSV files in a folder to Parquet format with progress bar.
    
    Args:
        csv_folder: folder containing CSV files
        parquet_folder: folder where Parquet files will be saved
    
    Returns:
        - Dictionary with success/error counts and file lists
    """
    csv_dir = Path(csv_folder)
    parquet_dir = Path(parquet_folder)
    
    # Create parquet folder if it doesn't exist
    parquet_dir.mkdir(exist_ok=True)
    
    # Get all CSV files
    csv_files = list(csv_dir.glob("*.csv"))
    
    if not csv_files:
        print(f"⚠️ No CSV files found in {csv_folder}")
        return None
    
    print(f"Found {len(csv_files)} CSV files to convert\n")
    
    success_files = []
    error_files = []
    
    # Process with progress bar
    for csv_file in tqdm(csv_files, desc="Converting CSV to Parquet"):
        try:
            df = pl.read_csv(csv_file, encoding="cp932")
            parquet_file = parquet_dir / csv_file.with_suffix('.parquet').name
            df.write_parquet(parquet_file)
            success_files.append(csv_file.name)
            
        except Exception as e:
            tqdm.write(f"❌ Error: {csv_file.name} - {e}")
            error_files.append(csv_file.name)
    
    # Summary
    print("\n" + "=" * 60)
    print(f"Conversion Complete!")
    print(f"✅ Success: {len(success_files)} files")
    print(f"❌ Errors: {len(error_files)} files")
    print("=" * 60)
    
    if error_files:
        print("\nFiles with errors:")
        for f in error_files:
            print(f"  - {f}")
    
    return {
        "success_count": len(success_files),
        "error_count": len(error_files),
        "success_files": success_files,
        "error_files": error_files
    }

In [9]:
convert_folder_to_parquet_with_progress("JAST_raw/dataset", parquet_folder="/Users/lex/CodeProjects/MyProject/Mitaron/Data/parquete_data")

Found 8 CSV files to convert



Converting CSV to Parquet: 100%|██████████| 8/8 [00:16<00:00,  2.01s/it]


Conversion Complete!
✅ Success: 8 files
❌ Errors: 0 files


{'success_count': 8,
 'error_count': 0,
 'success_files': ['dataset_000001_050000.csv',
  'dataset_100001_150000.csv',
  'dataset_200001_250000.csv',
  'dataset_275001_300000.csv',
  'dataset_150001_200000.csv',
  'dataset_250001_275000.csv',
  'dataset_050001_100000.csv',
  'dataset_300001_312573.csv'],
 'error_files': []}

### Testing

In [10]:
# Function for counting the unique number of variable in the folder:
def count_unique_infolder(folder_path, column):
    """Counting the Unique variables across the folders _

    Args:
        folder_path (str): path of the folder you want to find, make sure it is the parquet folder 
        column (str): the column through which you want to go

    Returns:
        pl.DataFrame: Table with filename and unique count
    """
    parquet_dir = Path(folder_path)
    parquet_files = list(parquet_dir.glob("*.parquet"))

    if not parquet_files:
        print(f"⚠️ No parquet files found in {parquet_files}")
        return None
    
    results = []
    for parquet in tqdm(parquet_files, desc="Checking Parquet"):
        try:
            df = pl.read_parquet(parquet)
            if column not in df.columns:
                tqdm.write(f"⚠️ Column {column} not found in {parquet.name}")
                continue
            n_unique = df[column].n_unique()
            results.append({"file": parquet.name, "n_unique": n_unique})
        except Exception as e:
            tqdm.write(f"❌ Error reading {parquet.name}: {e}")

    # Combining Results
    if not results:
        print("No results found")
        return None
    
    result_df = pl.DataFrame(results).sort("n_unique")

    print("\n✅ Unique value counts by file:")
    print(result_df)

    return result_df

In [7]:
from pathlib import Path
import polars as pl
from tqdm import tqdm

def count_areaid_infolder(folder_path, column="area_id"):
    """
    Counts how many occurrences of each area_id appear in each parquet file in a folder.

    Args:
        folder_path (str): Path to the folder containing parquet files.
        column (str): Column name to count (default: "area_id").

    Returns:
        pl.DataFrame: Table with file, area_id, and count.
    """
    parquet_dir = Path(folder_path)
    parquet_files = list(parquet_dir.glob("*.parquet"))

    if not parquet_files:
        print(f"⚠️ No parquet files found in {folder_path}")
        return None

    results = []

    for parquet in tqdm(parquet_files, desc="Counting area_id occurrences"):
        try:
            # Read each parquet file (eager for simplicity)
            df = pl.read_parquet(parquet)
            if column not in df.columns:
                tqdm.write(f"⚠️ Column {column} not found in {parquet.name}")
                continue

            # Count occurrences of each area_id
            counts = (
                df.group_by(column)
                  .agg(pl.count().alias("count"))
                  .with_columns(pl.lit(parquet.name).alias("file"))
            )

            # Append to results list
            results.append(counts)

        except Exception as e:
            tqdm.write(f"❌ Error reading {parquet.name}: {e}")

    # Combine all results into a single DataFrame
    if not results:
        print("No valid results found.")
        return None

    result_df = pl.concat(results)

    # Optional: sort results by file then area_id
    result_df = result_df.sort(["file", column])

    print("\n✅ Area ID counts by file:")
    print(result_df)

    return result_df

In [11]:
### Determining folder to use ###
folder_path = "/Users/lex/CodeProjects/MyProject/Mitaron/Data/parquete_data"
column = "patient_id"
###
result_df = count_unique_infolder(folder_path, column)


Checking Parquet: 100%|██████████| 8/8 [00:00<00:00, 20.14it/s]


✅ Unique value counts by file:
shape: (8, 2)
┌───────────────────────────────┬──────────┐
│ file                          ┆ n_unique │
│ ---                           ┆ ---      │
│ str                           ┆ i64      │
╞═══════════════════════════════╪══════════╡
│ dataset_300001_312573.parquet ┆ 12535    │
│ dataset_275001_300000.parquet ┆ 24965    │
│ dataset_250001_275000.parquet ┆ 24976    │
│ dataset_200001_250000.parquet ┆ 49802    │
│ dataset_000001_050000.parquet ┆ 49868    │
│ dataset_050001_100000.parquet ┆ 49888    │
│ dataset_100001_150000.parquet ┆ 49896    │
│ dataset_150001_200000.parquet ┆ 49929    │
└───────────────────────────────┴──────────┘


In [9]:
print(result_df["n_unique"].sum())

311859


### Fixing the Code names

In [ ]:
# from pathlib import Path
# import polars as pl
# from tqdm import tqdm

# def process_parquet_folder(input_folder: str, output_folder: str):
#     """
#     Converts multiple `area_` dummy columns into a single `area_id` column
#     for all .parquet files in a folder and saves them to a new folder.

#     Args:
#         input_folder (str): Path to the folder containing .parquet files.
#         output_folder (str): Path to the folder to save transformed files.
#     """
#     input_dir = Path(input_folder)
#     output_dir = Path(output_folder)
#     output_dir.mkdir(parents=True, exist_ok=True)

#     parquet_files = list(input_dir.glob("*.parquet"))
#     if not parquet_files:
#         print(f"⚠️ No parquet files found in {input_folder}")
#         return

#     print(f"Found {len(parquet_files)} parquet files to process.\n")

#     for file_path in tqdm(parquet_files, desc="Processing Parquet Files"):
#         try:
#             # Lazy scan
#             lf = pl.scan_parquet(file_path)

#             # Identify area columns
#             area_cols = [c for c in lf.columns if c.startswith("area_")]

#             if not area_cols:
#                 print(f"⚠️ No area_ columns found in {file_path.name}")
#                 continue

#             # Unpivot (wide → long)
#             lf_long = lf.unpivot(
#                 index=[c for c in lf.columns if c not in area_cols],
#                 on=area_cols,
#                 variable_name="area",
#                 value_name="visit_number"
#             )

#             # **CRITICAL: Filter to keep only rows where visit number > 0**
#             lf_long = lf_long.filter(pl.col("visit_number") >= 1)

#             # Create numeric area_id
#             lf_long = lf_long.with_columns(
#                 pl.col("area")
#                 .str.replace("area_", "")
#                 .cast(pl.Int32)  # Convert to integer
#                 .alias("area_id")
#             )

#             # Drop leftover columns
#             lf_result = lf_long.drop(["area"])

#             # Move area_id to 12th column if possible
#             cols = lf_result.columns
#             if "area_id" in cols:
#                 cols.remove("area_id")
#                 insert_pos = min(11, len(cols))
#                 cols.insert(insert_pos, "area_id")
#                 lf_result = lf_result.select(cols)

#             # Save to output folder (same filename)
#             output_file = output_dir / file_path.name
#             lf_result.sink_parquet(output_file)
            
#         except Exception as e:
#             print(f"\n❌ Error processing {file_path.name}: {str(e)}")
#             continue

#     print(f"\n✅ All files processed and saved to: {output_folder}")

# Newest Version of the Column Changes

In [86]:
from pathlib import Path
import polars as pl
from tqdm import tqdm

def process_parquet_folder(input_folder: str, output_folder: str):
    """
    Converts multiple `area_` columns into a single `area_id` column (primary/most visited area)
    for all .parquet files in a folder and saves them to a new folder.
    Also deletes specified unnecessary columns.

    Args:
        input_folder (str): Path to the folder containing .parquet files.
        output_folder (str): Path to the folder to save transformed files.
    """
    input_dir = Path(input_folder)
    output_dir = Path(output_folder)
    output_dir.mkdir(parents=True, exist_ok=True)

    parquet_files = list(input_dir.glob("*.parquet"))
    if not parquet_files:
        print(f"⚠️ No parquet files found in {input_folder}")
        return

    print(f"Found {len(parquet_files)} parquet files to process.\n")

    # Columns to delete
    deleting_cols = [
        "qualification_date", "disqualification_date", "shibou_flg",
        "bed_cnt_rank_0", "bed_cnt_rank_1_19", "bed_cnt_rank_20_199", "bed_cnt_rank_200"
    ]

    for file_path in tqdm(parquet_files, desc="Processing Parquet Files"):
        try:
            # Lazy scan
            lf = pl.scan_parquet(file_path)

            # Identify area columns
            area_cols = [c for c in lf.columns if c.startswith("area_")]

            if not area_cols:
                print(f"⚠️ No area_ columns found in {file_path.name}")
                continue

            # Get identifying columns (non-area columns)
            id_cols = [c for c in lf.columns if c not in area_cols]

            # Unpivot (wide → long)
            lf_long = lf.unpivot(
                index=id_cols,
                on=area_cols,
                variable_name="area",
                value_name="visit_number"
            )

            # Filter to keep only rows where visit number > 0
            lf_long = lf_long.filter(pl.col("visit_number") > 0)

            # Create numeric area_id
            lf_long = lf_long.with_columns(
                pl.col("area")
                .str.replace("area_", "")
                .cast(pl.Int32)
                .alias("area_id")
            )

            # Keep only the area with maximum visits per original row
            # In case of ties, keep the one with lowest area_id (tiebreaker)
            lf_result = (
                lf_long
                .sort(["visit_number", "area_id"], descending=[True, False])
                .group_by(id_cols)
                .first()
            )

            # Drop leftover columns
            lf_result = lf_result.drop(["area"])

            # 🔹 Drop unnecessary columns if they exist
            lf_result = lf_result.drop([c for c in deleting_cols if c in lf_result.columns])

            # Move area_id and visit_number next to each other (starting at 12th col if possible)
            cols = lf_result.columns
            if "area_id" in cols and "visit_number" in cols:
                cols.remove("area_id")
                cols.remove("visit_number")
                insert_pos = min(11, len(cols))
                cols.insert(insert_pos, "area_id")
                cols.insert(insert_pos + 1, "visit_number")
                lf_result = lf_result.select(cols)

            # Save to output folder (same filename)
            output_file = output_dir / file_path.name
            lf_result.sink_parquet(output_file)
            
        except Exception as e:
            print(f"\n❌ Error processing {file_path.name}: {str(e)}")
            continue

    print(f"\n✅ All files processed and saved to: {output_folder}")

In [87]:
input_folder = "/Users/lex/CodeProjects/MyProject/Mitaron/Data/parquete_data"
output_folder = "/Users/lex/CodeProjects/MyProject/Mitaron/Parquet_fresh"
process_parquet_folder(input_folder,output_folder)

Found 8 parquet files to process.



Processing Parquet Files:   0%|          | 0/8 [00:00<?, ?it/s]/var/folders/p_/j1bs4y9n44n99d120x17dn_00000gn/T/ipykernel_46122/4017328061.py:38: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  area_cols = [c for c in lf.columns if c.startswith("area_")]
/var/folders/p_/j1bs4y9n44n99d120x17dn_00000gn/T/ipykernel_46122/4017328061.py:45: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  id_cols = [c for c in lf.columns if c not in area_cols]
/var/folders/p_/j1bs4y9n44n99d120x17dn_00000gn/T/ipykernel_46122/4017328061.py:79: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potential


✅ All files processed and saved to: /Users/lex/CodeProjects/MyProject/Mitaron/Parquet_fresh


In [83]:
df_origin = pl.read_parquet("/Users/lex/CodeProjects/MyProject/Mitaron/Data/parquete_data/dataset_050001_100000.parquet")
df_new = pl.read_parquet("/Users/lex/CodeProjects/MyProject/Mitaron/Parquet_fresh/dataset_050001_100000.parquet")
print(df_new.shape, df_origin.shape)

(792873, 42) (792873, 79)


In [84]:
df_new

patient_no,insurer_cd,patient_id,sex_type_nm,birth_date,medtreat_yymm,rezept_family_type_nm,annual_salary_rank,business_type,public_expense_cd,ika_in_rz_cnt,area_id,ika_in_medtreat_days,ika_in_req_amt,ika_out_rz_cnt,ika_out_medtreat_days,ika_out_req_amt,sika_rz_cnt,sika_medtreat_days,sika_req_amt,cho_rz_cnt,cho_medtreat_days,cho_req_amt,sick_cd_cnt,sick_cd_main_cnt,sick_cd_utagai_cnt,outcome_type_tiyu,iy_amt,iy_std_cd7_cnt,yj_cd_cnt,iy_times_max,ge_rate,iy_min_amt_gap,iy_refill_times_1,iy_refill_times_2,iy_refill_times_3,ika_si_cd_cnt,sika_si_cd_cnt,sika_si_kasan_cnt,cho_si_cd_cnt,,visit_number
i64,i64,str,str,i64,i64,str,str,str,i64,i64,i32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,str,i64
78320,89,"""RI0007849804""","""男""",201807,202002,"""家族""","""750～799万円""","""化学工業・同類似業""",0,0,141003,0,0,2,2,11570,0,0,0,0,0,0,3,1,0,0,0,0,0,0,"""""",0,0,0,0,7,0,0,0,null,2
78377,89,"""RI0007970296""","""女""",201810,202211,"""家族""","""750～799万円""","""化学工業・同類似業""",0,0,141003,0,0,2,3,20020,0,0,0,0,0,0,4,0,0,1,30,2,2,1,"""""",0,0,0,0,7,0,0,0,null,2
78343,89,"""RI0007912597""","""男""",201201,201808,"""家族""","""750～799万円""","""化学工業・同類似業""",0,0,141003,0,0,2,2,13930,0,0,0,0,0,0,2,2,0,0,120,1,1,3,"""0.0000""",0,0,0,0,11,0,0,0,null,2
76963,89,"""RI0005070802""","""男""",201512,201810,"""家族""","""750～799万円""","""化学工業・同類似業""",1,1,141003,3,380160,1,1,16650,0,0,0,0,0,0,3,1,1,1,6000,10,12,1,"""0.0000""",0,0,0,0,53,0,0,0,null,2
65981,77,"""RI0003592469""","""男""",200910,201801,"""家族""","""750～799万円""","""その他""",2,1,141003,8,408870,1,2,6140,0,0,0,0,0,0,22,1,4,1,81190,21,22,10,"""0.2777""",870,0,0,0,77,0,0,0,null,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
90214,101,"""RI0006501615""","""女""",200201,201801,"""家族""","""750～799万円""","""その他""",0,0,342076,0,0,1,1,3290,0,0,0,0,0,0,1,1,0,0,1400,2,2,28,"""""",0,0,0,0,7,0,0,0,null,1
79791,91,"""RI0000429173""","""女""",200903,202108,"""家族""","""1250万円以上""","""金融業、保険業""",0,0,342076,0,0,1,1,2820,0,0,0,0,0,0,3,1,0,0,0,0,0,0,"""""",0,0,0,0,7,0,0,0,null,1
65094,77,"""RI0002047620""","""女""",201703,201909,"""家族""","""750～799万円""","""その他""",1,0,342076,0,0,1,5,24650,0,0,0,0,0,0,12,3,0,1,0,0,0,0,"""""",0,0,0,0,1,0,0,0,null,1


In [ ]:
deleting_cols = ["qualification_date", "disqualification_date", "shibou_flg", "bed_cnt_rank_0","bed_cnt_rank_1_19", "bed_cnt_rank_20_199", "bed_cnt_rank_200"]